In [9]:
import os
os.environ['PATH'] = "D:/108/openslide-win64-20171122/openslide-win64-20171122/bin" + ";" + os.environ['PATH']

# windows記得先載visual studio c++
import cv2
import json
# import openslide
import matplotlib.pyplot as plt
import large_image
import numpy as np
import skimage.io
from skimage import io
# import large_image_source_openslide
from PIL import Image
# from openslide.deepzoom import DeepZoomGenerator
import matplotlib.image as mpimg
from tqdm import tqdm_notebook
import math
import pandas as pd
import MTM  # multi template matching
from MTM import matchTemplates, drawBoxesOnRGB
from shapely.geometry import Polygon
from scipy import stats
import glob
from pandas import json_normalize

Image.MAX_IMAGE_PIXELS = 10000000000

In [38]:
o = cv2.imread("./all_image/BFC/origin/BFC_20_001_0.png")
gray = cv2.cvtColor(o , cv2.COLOR_BGR2GRAY)
mean = []
# for i in range(len(gray)):
#     if round(gray[i].mean()) <= gray.mean():
#         mean.append(round(gray[i].mean()))
# mode = stats.mode(mean)
# if mode[0][0] >= 180:
ret , binary = cv2.threshold(gray , 150 , 255 , cv2.THRESH_BINARY) # ret = threshold
contours , hierarchy = cv2.findContours(binary , cv2.RETR_LIST , cv2.CHAIN_APPROX_SIMPLE)
#                 plt.imshow(o)
#                 plt.show()
for i in range(len(contours)):
    if len(contours[i]) >= 20 and len(contours[i]) <= 60 :
        ellipse = cv2.fitEllipse(contours[i])
        if 18 < ellipse[1][0] < 40 and 18 < ellipse[1][1] < 40: 
            # print(image_name.split(".png")[0] + "_" + str(i),"的 ellipse = " , ellipse[1])
            result_ellipse = cv2.ellipse(o , ellipse , (255,255,255))

            # get the parameter of the small rectangle
            center, size, angle = ellipse[0], ellipse[1], ellipse[2]
            center, size = tuple(map(int, center)), tuple(map(int, size))

            # get row and col num in img
            height, width = result_ellipse.shape[0], result_ellipse.shape[1]

            # calculate the rotation matrix
            M = cv2.getRotationMatrix2D(center, angle, 1)
            # rotate the original image
            img_rot = cv2.warpAffine(result_ellipse, M, (width, height))

            # now rotated rectangle becomes vertical and we crop it
            img_crop = cv2.getRectSubPix(img_rot, size, center)
            
            test = []
            for j in contours[i]:
                test.append(j[0])
            x_max = max(test, key=lambda item: item[0])
            y_max = max(test, key=lambda item: item[1])
            x_min = min(test, key=lambda item: item[0])
            y_min = min(test, key=lambda item: item[1])
            x = int(x_min[0])
            y = int(y_min[1])
            w = int(x_max[0] - x_min[0])+1
            h = int(y_max[1] - y_min[1])+1
            print(f'{i},{x_max},{x_min},{y_max},{y_min},{[x,y,w,h]}')

0,[ 83 229],[ 54 233],[ 69 241],[ 66 216],[54, 216, 30, 26]
1,[128 216],[100 224],[113 235],[121 210],[100, 210, 29, 26]
2,[161 196],[132 204],[145 212],[148 188],[132, 188, 30, 25]
4,[ 61 195],[ 31 190],[ 52 209],[ 41 183],[31, 183, 31, 27]
6,[ 96 190],[ 66 190],[ 88 204],[ 86 176],[66, 176, 31, 29]
9,[ 48 164],[ 16 169],[ 35 184],[ 28 156],[16, 156, 33, 29]
10,[ 93 161],[ 61 165],[ 77 175],[ 72 148],[61, 148, 33, 28]
14,[118 138],[ 90 145],[101 155],[100 129],[90, 129, 29, 27]
15,[159 135],[130 143],[145 154],[146 125],[130, 125, 30, 30]
18,[226  91],[202  96],[217 111],[215  81],[202, 81, 25, 31]


IndexError: index 1 is out of bounds for axis 0 with size 1